In [1]:
import os
import pandas as pd
import numpy as np
import cv2
from PIL import Image, ImageDraw

import matplotlib.pyplot as plt
import seaborn as sns
import plotly

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
ENV_DIR = '../input'
DATA_DIR = f'{ENV_DIR}/nfl-health-and-safety-helmet-assignment'

In [4]:
os.listdir(DATA_DIR)

In [5]:
# Training data
# -----------------------------------------------------------------------
# Player information is included

# Bounding Box
train_df = pd.read_csv(f'{DATA_DIR}/train_labels.csv')

# Tracking Information using Sensor
train_tracking_df = pd.read_csv(f'{DATA_DIR}/train_player_tracking.csv')
test_tracking_df = pd.read_csv(f'{DATA_DIR}/test_player_tracking.csv')

# images/
# -----------------------------------------------------------------------
# Trained images using images_labels.csv and predict the train, test
# The prediction result is [train/test]_baseline_helmets.csv
# No player information is included

# information of images without player information
image_df = pd.read_csv(f'{DATA_DIR}/image_labels.csv')

# Baseline Prediction - Trained by images inside folder images/
train_predict_df = pd.read_csv(f'{DATA_DIR}/train_baseline_helmets.csv')

test_predict_df = pd.read_csv(f'{DATA_DIR}/test_baseline_helmets.csv')

In [6]:
# reference : https://www.kaggle.com/coldfir3/eda-helmet-keypoint-tracking-data-comparison
def get_frame_from_video(video_path, frame):
    video_path = f"{DATA_DIR}/train/{video_path}"
    frame = frame - 1
    
    !ffmpeg \
        -hide_banner \
        -loglevel fatal \
        -nostats \
        -i $video_path -vf "select=eq(n\,$frame)" -vframes 1 frame.png
    
    img = Image.open('frame.png')
    os.remove('frame.png')
    return img

In [7]:
def draw_rect(image, bbox_df):
    new_image = image.copy()
    draw = ImageDraw.Draw(new_image)
    for _, (left, width, top, height) in bbox_df[['left', 'width', 'top', 'height']].iterrows():
        draw.rectangle(((left, top), (left + width, top + height)), outline=(255, 0, 0), width=2)
    
    return new_image

def frame_bbox(df, video_frame):
    video_name = '_'.join(video_frame.split('_')[:3]) + '.mp4'
    frame = int(video_frame.split('_')[-1])
    
    image = get_frame_from_video(video_name, frame)
    bbox_df = df.query('video_frame == @video_frame')
    
    bbox_image = draw_rect(image, bbox_df)
    
    return bbox_image

In [8]:
from IPython.display import Video, display

def video(video_path, ratio=0.7):
    nfl_video = Video(f"{DATA_DIR}/train/{video_path}",
                      embed=True,
                      height=int(720 * ratio),
                      width=int(1280 * ratio))
    return nfl_video
    
video('57583_000082_Endzone.mp4')

In [9]:
# Reference : https://www.kaggle.com/robikscube/nfl-helmet-assignment-getting-started-guide
def add_track_features(tracks, fps=59.94, snap_frame=10):
    """
    Add column features helpful for syncing with video data.
    """
    tracks = tracks.copy()
    tracks["game_play"] = (
        tracks["gameKey"].astype("str")
        + "_"
        + tracks["playID"].astype("str").str.zfill(6)
    )
    tracks["time"] = pd.to_datetime(tracks["time"])
    
    # The time when snap happened
    snap_dict = (
        tracks.query('event == "ball_snap"')
        .groupby("game_play")["time"]
        .first()
        .to_dict()
    )
    tracks["snap"] = tracks["game_play"].map(snap_dict)
    tracks["isSnap"] = tracks["snap"] == tracks["time"]
    tracks["team"] = tracks["player"].str[0].replace("H", "Home").replace("V", "Away")
    tracks["snap_offset"] = (tracks["time"] - tracks["snap"]).astype(
        "timedelta64[ms]"
    ) / 1_000
    # Estimated video frame
    tracks["est_frame"] = (
        ((tracks["snap_offset"] * fps) + snap_frame).round().astype("int")
    )
    return tracks

train_tracking_df = add_track_features(train_tracking_df)

In [10]:
import plotly.express as px
import plotly.graph_objects as go
import plotly


def add_plotly_field(fig):
    # Reference https://www.kaggle.com/ammarnassanalhajali/nfl-big-data-bowl-2021-animating-players
    fig.update_traces(marker_size=20)
    #update_traces -> https://plotly.com/python/creating-and-updating-figures/#updating-traces
    
    fig.update_layout(paper_bgcolor='#29a500', plot_bgcolor='#29a500', font_color='white',
        width = 800,
        height = 600,
        title = "",
        
        xaxis = dict(
        nticks = 10,
        title = "",
        visible=False
        ),
        yaxis = dict(
        scaleanchor = "x",
        title = "Temp",
        visible=False
        ),
        showlegend= True,
  
        annotations=[
       dict(
            x=-5,
            y=26.65,
            xref="x",
            yref="y",
            text="ENDZONE",
            font=dict(size=16,color="#e9ece7"),
            align='center',
            showarrow=False,
            yanchor='middle',
            textangle=-90
        ),
        dict(
            x=105,
            y=26.65,
            xref="x",
            yref="y",
            text="ENDZONE",
            font=dict(size=16,color="#e9ece7"),
            align='center',
            showarrow=False,
            yanchor='middle',
            textangle=90
        )]  
        ,
        legend=dict(
        traceorder="normal",
        font=dict(family="sans-serif",size=12),
        title = "",
        orientation="h",
        yanchor="bottom",
        y=1.00,
        xanchor="center",
        x=0.5
        ),
    )
    ####################################################
        
    fig.add_shape(type="rect", x0=-10, x1=0,  y0=0, y1=53.3,line=dict(color="#c8ddc0",width=3),fillcolor="#217b00" ,layer="below")
    fig.add_shape(type="rect", x0=100, x1=110, y0=0, y1=53.3,line=dict(color="#c8ddc0",width=3),fillcolor="#217b00" ,layer="below")
    for x in range(0, 100, 10):
        fig.add_shape(type="rect", x0=x,   x1=x+10, y0=0, y1=53.3,line=dict(color="#c8ddc0",width=3),fillcolor="#29a500" ,layer="below")
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=1, x1=x, y1=2,line=dict(color="#c8ddc0",width=2),layer="below")
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=51.3, x1=x, y1=52.3,line=dict(color="#c8ddc0",width=2),layer="below")
    
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=20.0, x1=x, y1=21,line=dict(color="#c8ddc0",width=2),layer="below")
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=32.3, x1=x, y1=33.3,line=dict(color="#c8ddc0",width=2),layer="below")
    
    
    fig.add_trace(go.Scatter(
    x=[2,10,20,30,40,50,60,70,80,90,98], y=[5,5,5,5,5,5,5,5,5,5,5],
    text=["G","1 0","2 0","3 0","4 0","5 0","4 0","3 0","2 0","1 0","G"],
    mode="text",
    textfont=dict(size=20,family="Arail"),
    showlegend=False,
    ))
    
    fig.add_trace(go.Scatter(
    x=[2,10,20,30,40,50,60,70,80,90,98], y=[48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3],
    text=["G","1 0","2 0","3 0","4 0","5 0","4 0","3 0","2 0","1 0","G"],
    mode="text",
    textfont=dict(size=20,family="Arail"),
    showlegend=False,
    ))
    
    return fig

In [11]:
train_tracking_df

In [12]:
def football_animation(game_play):
    train_tracking_df["track_time_count"] = (
        train_tracking_df.sort_values("time")
        .groupby("game_play")["time"]
        .rank(method="dense")
        .astype("int")
    )

    fig = px.scatter(
        train_tracking_df.query("game_play == @game_play"),
        x="x",
        y="y",
        range_x=[-10, 110],
        range_y=[-10, 53.3],
        hover_data=["player", "s", "a", "dir"],
        color="team",
        animation_frame="track_time_count",
        text="player",
        title=f"Animation of NGS data for game_play {game_play}",
    )

    fig.update_traces(textfont_size=10)
    fig = add_plotly_field(fig)
    fig.show()

In [13]:
football_animation('57583_000082')

In [14]:
train_videos = os.listdir(f'{DATA_DIR}/train')
test_videos = os.listdir(f'{DATA_DIR}/test')

len(train_videos), len(test_videos)

In [15]:
set(test_videos).issubset(set(train_videos))

In [16]:
end_count = 0
side_count = 0
endzone_list = []
sideline_list = []
for train_video in train_videos:
    name = train_video.split('.')[0]
    video_id, play_id, view = name.split('_')
    
    if view == "Endzone":
        endzone_list.append('_'.join([video_id, play_id]))
        end_count += 1
    else:
        sideline_list.append('_'.join([video_id, play_id]))
        side_count += 1

print(end_count, side_count)

In [17]:
len(set(endzone_list)), len(set(sideline_list)), set(endzone_list) == set(sideline_list)

In [18]:
#videos inside folder matches the video list inside the train_labels.csv
set(train_videos) == set(train_df.video.unique())

In [19]:
not_match_video = []

for play_id in train_df.playID.unique():
    end_frame_n = train_df.query('playID == @play_id and view == "Endzone"').frame.max()
    side_frame_n = train_df.query('playID == @play_id and view == "Sideline"').frame.max()
    
    if end_frame_n != side_frame_n:
        not_match_video.append(play_id)
        print(f'Not same at playID {play_id} endzone [{end_frame_n}] sideline [{side_frame_n}] difference [{abs(end_frame_n - side_frame_n)}]')

In [20]:
len(not_match_video)

In [21]:
def get_total_frame(video_path):
    cap = cv2.VideoCapture(f"{DATA_DIR}/train/{video_path}")
    property_id = int(cv2.CAP_PROP_FRAME_COUNT)
    length = int(cv2.VideoCapture.get(cap, property_id))
    
    return length

In [22]:
play2frame = train_df.groupby("video").frame.max().to_dict()

In [23]:
frame_df = train_df.query("video == '57584_000336_Sideline.mp4'")
frame_df

In [24]:
frame_df.frame.max()
get_total_frame("57584_000336_Sideline.mp4")

In [25]:
test_df = train_df.query("video in @test_videos").reset_index().copy()
test_df

In [26]:
train_df.groupby("gameKey")["playID"].unique()

In [27]:
train_df.groupby(["gameKey", "playID", "view"])["frame"].nunique()

In [28]:
sns.displot(train_df.groupby(["gameKey", "playID", "view"])["frame"].nunique().values);

In [29]:
train_df.groupby(['gameKey', 'playID', 'view'])['frame'].nunique().sum()

In [30]:
play_per_game = train_df.groupby('gameKey')['playID'].nunique().reset_index().groupby('playID')['gameKey'].unique().to_dict()
play_per_game

In [31]:
train_df.groupby(['gameKey', 'playID', 'view'])['label'].nunique().value_counts()

In [32]:
# check what game does only 16 players are running for?
train_df.groupby(['gameKey', 'playID', 'view'])['label'].nunique().reset_index().query('label == 16')

In [33]:
sideline_df = train_df.groupby(['gameKey', 'playID', 'view'])['label', 'isSidelinePlayer'].nunique().query('isSidelinePlayer == 2')
sideline_df

In [34]:
sideline_df.reset_index().view.value_counts()

In [35]:
sideline_df.label.value_counts()

# What is the helmet size inside the video?
* 5928 is the biggest helmet size shown in the video and occure when the camera is zooming.
* 9 is the smallest helmets size
* Mostly the helmet size are around 150

In [36]:
train_df["helmet_size"] = train_df.width * train_df.height
train_df.helmet_size.hist();

In [37]:
train_df.helmet_size.value_counts()[:10]

In [38]:
train_df.helmet_size.max(), train_df.helmet_size.min()

In [39]:
train_df.query("helmet_size == 5928")

In [40]:
# when does the helmet shown the biggest?
get_frame_from_video('57686_002546_Endzone.mp4', 429)

In [41]:
# check through the video
video('57686_002546_Endzone.mp4')

In [42]:
train_df.query("helmet_size == 9")

In [43]:
# when does the helmet shown the smallest?
get_frame_from_video('57680_002206_Sideline.mp4', 149)

In [44]:
# check through the video
video('57680_002206_Sideline.mp4')

# Is Definitive Impact related to Impact type?
| Yes, Definitive impact is subset of impacts and all types of impact could be definitive impact

In [45]:
train_df.impactType.value_counts()

In [46]:
impact_index = train_df.query("impactType != 'None'").index
impact_index

In [47]:
definite_impact_index = train_df.query("isDefinitiveImpact == True").index
definite_impact_index

In [48]:
set(impact_index).issubset(set(definite_impact_index)), set(definite_impact_index).issubset(set(impact_index))

In [49]:
train_df.query("isDefinitiveImpact == True").impactType.value_counts()

# How often Definitive Impact Happens?
* Definitive impact moments is  500 times smaller than normal impact

In [50]:
train_df.isDefinitiveImpact = train_df.isDefinitiveImpact.astype(int)
train_df.isDefinitiveImpact.head(1)

In [51]:
train_df.isDefinitiveImpact.value_counts()

In [52]:
950198 / 1889

# [Train/Test]palyer tracking.csv
| Tracking information of the players that is used with videos to map the helmet label
The associated test_player_tracking.csv are available to your model when submitting.
So we will not consider test_player_tracking.csv here


Questions

How many games, play, frame?1

Is ball snap is the starting point of the game??2

Does track information always track 22 players every moment?3

Why only 11 players are tracked?4

Does tracking information always recorded longer than video?5

When event happens were there always 22 players?6

Does all trackings recorded before the ball snap?7

What plays has unconsistent player numbers while beeing tracked?8

Does player number unconsistent even we only consider the time period of train videos?9

test_tracking_df is not shown until submission?

# How many games, play, frame?
* 50 games
* 60 plays
* 15180 frames (frames means images)
    * min113
    * max456

In [53]:
train_tracking_df.gameKey.nunique()

In [54]:
train_tracking_df.playID.nunique()

In [55]:
frame_df = train_tracking_df.groupby(["gameKey", "playID"])["time"].nunique().reset_index()
frame_df

In [56]:
sns.displot(frame_df.time);

In [57]:
frame_df.time.sum()

In [58]:
frame_df.time.min(), frame_df.time.max()

# Is ball snap is the starting point of the game?
 > yes! It occurs only one time per game
 > * before snap players change their position a bit 
 > * After snap they start to run!

In [59]:
len(train_tracking_df.query("event == 'ball_snap'")) / 22

In [60]:
train_tracking_df.query("event == 'ball_snap'").playID.value_counts() / 22

In [61]:
train_tracking_df.query("playID == 82 & event == 'ball_snap'").head(1)

In [62]:
# We can see that the player is not moving while 15 seconds flow
train_tracking_df.query('playID == 82 and snap_offset < 0 and player == "H97"')

In [63]:
before_snap_df = train_tracking_df.query('playID == 82 and snap_offset < 0 and player == "H97"')
before_snap_df.s.min(), before_snap_df.s.max()

In [64]:
# before the ball snap the movement is Low
sns.displot(before_snap_df.s);

In [65]:
# Mostly 22 players are tracked but not all
train_tracking_df.groupby(['playID', 'time']).count()['player'].value_counts()

# Why only 11 players are tracked?
> There were 22 players before the last frame and  suddenly became 11 players.
> This seems not right but 11 players tracking information shown after the video ended so doesn't need to be considered.

In [66]:
player_n_df = train_tracking_df.groupby(['playID', 'time']).count()['player'].reset_index()
player_n_df

In [67]:
player_n_df.query("player == 11")

In [68]:
# There are 11 players at the last frame
train_tracking_df.query('time == "2018-10-29 02:22:44.099000+00:00"')

In [69]:
video("57686_002546_Sideline.mp4")

In [70]:
football_animation("57686_002546")

# Does tracking information always recorded longer than video?
> yes, all the track time is longer than train videos and mostly 3 times longer! Some tracking informatin is approximately 6 times longer!

In [71]:
time_df = ((train_df.groupby(['playID', 'view']).frame.max() - train_df.groupby(['playID', 'view']).frame.min()) / 59.94).reset_index()
time_df = time_df.rename(columns={'frame': 'time_cost'})
time_df

In [72]:
track_time_dict = (train_tracking_df.groupby('playID').snap_offset.max() - train_tracking_df.groupby('playID').snap_offset.min()).to_dict()
track_time_dict

In [73]:
time_df["track_time_cost"] = time_df.playID.map(track_time_dict)
all(time_df["time_cost"] < time_df["track_time_cost"])

In [74]:
#It's mostly 3 times longer than train videos
sns.displot(time_df["track_time_cost"]/time_df["time_cost"]);

# When event happens were there always 22 players?
> Yes! But these seems to be just luck that all 22 players were on the ground when event happens

In [75]:
event_df = (train_tracking_df.groupby(["playID", "event"])["time"].count()/22).reset_index()
event_df

In [76]:
event_df.event.value_counts()

In [77]:
event_df.time.value_counts()

# Does all trackings recorded before the ball snap?
> Yes! All plays are recorded before the ball snap!

In [78]:
len(train_tracking_df.query("snap_offset < 0").playID.unique())

# What plays has unconsistent player numbers while beeing tracking?
>there are 6 plays that are not consistent.
> * The ID is 109, 336, 350, 1242, 2546, 4152

In [79]:
player_df = train_tracking_df.groupby(["playID", "time"]).count().player.reset_index()
player_df

In [80]:
player_df = train_tracking_df.groupby(["playID", "time"]).count()
player_df

In [81]:
history_df = player_df.groupby("playID").apply(lambda r: r["player"].values).reset_index()
history_df = history_df.rename(columns = {0: "history"})
history_df.head()

In [82]:
not_consistent = []
for _, (play_id, history) in history_df.iterrows():
    start_player_n = history[0]
    all_same = all(history == start_player_n)
    
    if not all_same:
        not_consistent.append(play_id)
not_consistent

# Does player number inconsistent even we only consider the time period of train videos?
> It become more consistent when considering only the time period of train videos. But still 3 tracking information is not consistent. **We need to think how to impute this missing data.**
> * Considering the whole tracking - 109, 336, 350, 1242, 2546, 4152
> * Only for the video time period - 109, 336, 4152

In video time period 21 players are always tracked and 1 player is not tracked in some moments.

In [83]:
min_dict = train_df.groupby("playID").frame.min().to_dict()
max_dict = train_df.groupby("playID").frame.max().to_dict()
train_tracking_df["min_frame"] = train_tracking_df.playID.map(min_dict)
train_tracking_df["max_frame"] = train_tracking_df.playID.map(max_dict)
filter_df = train_tracking_df[["playID", "time", "player", "est_frame", "min_frame", "max_frame"]].copy()
filter_df

In [84]:
filter_df["inVideo"] = (filter_df.est_frame >= filter_df.min_frame) & (filter_df.est_frame <= filter_df.max_frame)
filter_df

In [95]:
filter_df = filter_df.query("inVideo == True")
filter_df

In [96]:
player_df = filter_df.groupby(['playID', 'time']).count().player.reset_index()
player_df

In [97]:
history_df = player_df.groupby('playID').apply(lambda r: r['player'].values).reset_index()
history_df = history_df.rename(columns={0: 'history'})
history_df.head()

In [98]:
not_consistent = []
for _, (play_id, history) in history_df.iterrows():
    start_player_n = history[0]
    all_same = all(history == start_player_n)
   
    if not all_same:
        not_consistent.append(play_id)
not_consistent

In [99]:
history_df.query("playID == 109").history.values

In [100]:
history_df.query("playID == 336").history.values

In [101]:
history_df.query("playID == 4152").history.values

In [103]:
video('57586_004152_Sideline.mp4')

In [104]:
football_animation('57586_004152')

# test_tracking_df is not shown until submission
> As data description says test_player_tracking.csv are available to your model when submitting.

In [107]:
test_tracking_df.gameKey.nunique()

In [108]:
test_tracking_df

# image_labels.csv
> image labels are supplement dataset that has various plays that contains mostly only 1 frame each. Only 2 frames match with train set. Not sure this could be used for training but surely could be used for training the bbox coordinate.

Comparation between `image_labels.csv` and `train_label.csv`

* 9947 supplement images vs 52142 train images
    * additional 20% data for bbox prediction only
* 41 games matches and only 1 plays matches
* only 2 frame matches

In [109]:
# number of bounding boxes of all frames in
len(image_df)

In [111]:
# 9947 images
image_df.image.nunique()

In [112]:
# This feature doesn't seem useful to our competition
image_df.label.hist();

In [113]:
image_df.groupby("image").count().head(5)

In [114]:
# remove "frame", ".jpg" to compare with the whole frame
image_df.image = image_df.image.str.replace("frame", "")
image_df.image = image_df.image.str.replace(".jpg", "")
image_df.head(5)


In [118]:
# additional, auxiliary images
aux_images = image_df.image

# How many games match?
> 41 gfames match.
>* 38 games that matches one play
>* 3 games that matches has two plays

In [119]:
aux_games_set = set(aux_images.str.split("_").str[0])
train_games_set = set(train_df.gameKey.astype(str))

In [126]:
# aux game ID 57502 ~ 58176
# train game ID 57583 ~ 58107
len(aux_games_set), len(train_games_set)

In [127]:
common_games = list(aux_games_set & train_games_set)
print(common_games)

In [128]:
# 38 games matches between one play only games and common games
one_play_only = set([str(game) for game in play_per_game[1]])
len(set(common_games).intersection(one_play_only))

In [137]:
# 3 two play only games matches with common games
two_play_only = set([str(game) for game in play_per_game[2]])
len(set(common_games).intersection(two_play_only))

# How many palys match?
> Only 1 matches. `58000_001306`. Besides `image_labels.csv` has lot's of various plays!

In [139]:
train_df

In [141]:
aux_play_set = set(aux_images.str.replace("_\d*$", ""))
train_play_set = set(train_df.video_frame.str.replace("_\d*$", ""))
len(aux_play_set), len(train_play_set)

In [145]:
aux_play_set & train_play_set

# How many frames match?
> Only 2 matches. `image_label.csv` has mostly one image per play

In [147]:
aux_frame_set = set(aux_images)
train_frame_set = set(train_df.video_frame)
len(aux_frame_set), len(train_frame_set)

In [148]:
# both are not subset for each other
aux_frame_set.issubset(train_frame_set), train_frame_set.issubset(aux_frame_set)

In [151]:
aux_frame_set & train_frame_set

# [Train/Test] baseline helmets.csv
> Just to check does the inference really executed under our train/test folder videos. And yes it did! After training the model on the supplement images, the model was used to predict the bbox of train/test videos.
>* if the bbox prediction is good with just supplement videos how great will it be when we train with our train set.

`test_tracking_df` is not shown until submission
As data description says **test_baseline_helmets.csv are available to your model when submitting.**

In [152]:
train_predict_df["video"] = train_predict_df.video_frame.str.replace("_\d*$", "")
test_predict_df["video"] = test_predict_df.video_frame.str.replace("_\d*$", "")
train_predict_df.head(5)

In [153]:
# prediction is for the train set that we are using
set(train_predict_df.video.unique()) == set(train_df.video_frame.str.replace("_\d*$", ""))

In [154]:
set(test_predict_df.video.unique())

In [155]:
test_videos.sort()
test_videos

# How many bbox was predicted for 1 frame?
> The model used for predicting the bounding box predicts sideline person head as helmet too

* 82 bbox was predicted in max for 1 frame
* 2 bbox was predicted in min for 1 frame

In [156]:
# 82 bbox was predicteed in max for 1 frame
train_predict_df.groupby("video_frame").count()["left"].max()

In [158]:
# bbox was predicted in max for 1 frame
train_predict_df.groupby("video_frame").count()["left"].min()

In [159]:
# bbox number doesn't seem quite right
sns.displot(train_predict_df.groupby("video_frame").count()["left"]);

In [160]:
train_predict_df.groupby("video_frame").count().query("left == 82")

In [161]:
frame_bbox(train_predict_df, "58094_002819_Sideline_169")

In [162]:
train_predict_df.groupby("video_frame").count().query("left == 2")

In [163]:
frame_bbox(train_predict_df, "57584_000336_Sideline_441")

# What if we remove prediction with lower confidence?
> Max bbox shrimp to 35 and the bbox prediction of false positive'on the sideline players seems more better than before! But still a long way to go!

In [164]:
filter_df = train_predict_df.copy().query("conf > 0.75")
filter_df

In [165]:
# 82 bbox was predicted in max for 1 frame
filter_df.groupby("video_frame").count()["left"].max()

In [166]:
frame_bbox(filter_df, '58094_002819_Sideline_169')

In [167]:
frame_bbox(filter_df, '57584_000336_Sideline_454')